### dev_notebook_

This is your development environment...you may start:
- Exploring your data.
- Testing the ad-hoc methods.
- Designing your pipeline.

In [2]:
import pandas as pd
import requests
from shapely.geometry import Point
import geopandas as gpd
from geopy.distance import geodesic as gd
import re
import numpy as np
from datetime import datetime

In [3]:
interest_url = '/catalogo/300356-0-monumentos-ciudad-madrid.json'
str(datetime.now()).split('.')[0].replace(':','').replace(' ','_')

'2023-11-13_185956'

In [4]:
def interest_points(url):
    base_url = 'https://datos.madrid.es/egob'
    df = pd.json_normalize(requests.get(base_url + url).json()['@graph'])
    print('Interest points dataframe created')
    return df

In [5]:
interest_df = interest_points(interest_url)
interest_df.head(10)

Interest points dataframe created


,@id,id,title,relation,references,address.district.@id,address.locality,address.postal-code,address.street-address,location.latitude,location.longitude,organization.organization-desc,organization.organization-name,address.area.@id
0,https://datos.madrid.es/egob/catalogo/tipo/mon...,409634,A las víctimas del Holocausto,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,JARDIN TRES CULTURAS,40.464322,-3.603558,"La obra, creada por el escultor judío de orige...",A las víctimas del Holocausto,NaN
1,https://datos.madrid.es/egob/catalogo/tipo/mon...,409512,A los Abuelos,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,C Alicún,40.479812,-3.661719,"Pequeña escultura, situada en un parque delimi...",A los Abuelos,https://datos.madrid.es/egob/kos/Provincia/Mad...
2,https://datos.madrid.es/egob/catalogo/tipo/mon...,408781,A Neruda-El Ser alado,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,Gta José Luis Ozores 116,40.381855,-3.652419,Monumento dedicado al escritor Pablo Neruda (1...,A Neruda-El Ser alado,https://datos.madrid.es/egob/kos/Provincia/Mad...
3,https://datos.madrid.es/egob/catalogo/tipo/mon...,408836,Abogados Laboristas,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,C Atocha 55,40.412911,-3.700017,El 24 de enero de 1977 se produjo un atentado ...,Abogados Laboristas,https://datos.madrid.es/egob/kos/Provincia/Mad...
4,https://datos.madrid.es/egob/catalogo/tipo/mon...,408920,Abstracta I,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,,40.421746,-3.662293,"La histórica quinta de la Fuente del Berro, de...",Abstracta I,https://datos.madrid.es/egob/kos/Provincia/Mad...
5,https://datos.madrid.es/egob/catalogo/tipo/mon...,408921,Abstracta II,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,,40.421962,-3.662260,"La histórica quinta de la Fuente del Berro, de...",Abstracta II,https://datos.madrid.es/egob/kos/Provincia/Mad...
6,https://datos.madrid.es/egob/catalogo/tipo/mon...,400147,Abundancia,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28003,CALLE GUZMAN EL BUENO 137,40.444201,-3.712595,En el año 1958 se celebró un sorteo extraordin...,Abundancia,https://datos.madrid.es/egob/kos/Provincia/Mad...
7,https://datos.madrid.es/egob/catalogo/tipo/mon...,414012,Academia de Farmacia,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,C Farmacia 9,40.423967,-3.699719,La Academia de Farmacia tuvo como antecedente ...,Academia de Farmacia,https://datos.madrid.es/egob/kos/Provincia/Mad...
8,https://datos.madrid.es/egob/catalogo/tipo/mon...,409550,Acueducto y rejas sobre el arroyo del Barranco...,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,CAMINO HUMERA,40.411744,-3.778890,"Esta construcción, híbrida de puente-acueducto...",Acueducto y rejas sobre el arroyo del Barranco...,NaN
9,https://datos.madrid.es/egob/catalogo/tipo/mon...,400108,Ad Iesum per Mariam,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28043,AVENIDA MACHUPICHU 1,40.455453,-3.623525,El grupo escultórico c

In [119]:
interest_df['Type of place'] = 'Monument'
interest_df=interest_df[['title', 'Type of place', 'address.street-address']]
interest_df.columns = ['Place of interest', 'Type of place', 'Place address']
interest_df

KeyError: "['title', 'address.street-address'] not in index"

In [3]:
bicimad_df2 = pd.read_csv('../data/origin/bicimad_stations.csv', sep='\t')
bicimad_df2.head(12)

,Unnamed: 0,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates
0,0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]"
1,1,2,1b - Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.701602938060457, 40.41731271011562]"
2,2,3,2 - Miguel Moya,3,2,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,"[-3.7058415, 40.4205886]"
3,3,4,3 - Plaza Conde Suchil,2,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,"[-3.7069171, 40.4302937]"
4,4,5,4 - Malasaña,1,4,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,"[-3.7025875, 40.4285524]"
5,5,6,5 - Fuencarral,1,5,Calle Fuencarral nº 108,1,0,27,25,0,0,Point,"[-3.70205, 40.42852]"
6,6,7,6 - Colegio Arquitectos,3,6,Calle Hortaleza nº 63,1,1,24,0,0,0,Point,"[-3.698447, 40.424148]"
7,7,8,7 - Hortaleza,3,7,Calle Hortaleza nº 75,1,1,21,0,0,0,Point,"[-3.6977715, 40.4251906]"
8,8,9,8 - Alonso Martínez,1,8,Plaza de Alonso Martínez nº 5,1,0,24,22,0,0,Point,"[-3.6954403, 40.4278682]"
9,9,10,9 - Plaza de San Miguel,2,9,Plaza de San Miguel nº 9,1,0,24,10,10,0,Point,"[-3.7095084, 40.4156057]"


In [96]:
bicipark_df = pd.read_csv('../data/origin/bicipark_stations.csv', sep=';')
bicipark_df.loc[2,['geometry.coordinates']]

geometry.coordinates    [-3.690786, 40.418146]
Name: 2, dtype: object

In [98]:
bicipark_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            11 non-null     int64 
 1   stationId             11 non-null     int64 
 2   stationName           11 non-null     object
 3   address               11 non-null     object
 4   city                  11 non-null     object
 5   zip_code              11 non-null     object
 6   total_places          11 non-null     int64 
 7   free_places           11 non-null     int64 
 8   enabled               11 non-null     int64 
 9   reserved_places       11 non-null     int64 
 10  geometry.type         11 non-null     object
 11  geometry.coordinates  11 non-null     object
dtypes: int64(6), object(6)
memory usage: 1.2+ KB


In [4]:
bicimad_df2[['longitude', 'latitude']] = bicimad_df2['geometry.coordinates'].str.strip('[]').str.split(',',expand= True).astype('float64')
bicimad_df2['longitude'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 264 entries, 0 to 263
Series name: longitude
Non-Null Count  Dtype  
--------------  -----  
264 non-null    float64
dtypes: float64(1)
memory usage: 2.2 KB


In [5]:
bicimad_df2.columns

Index(['Unnamed: 0', 'id', 'name', 'light', 'number', 'address', 'activate',
       'no_available', 'total_bases', 'dock_bikes', 'free_bases',
       'reservations_count', 'geometry.type', 'geometry.coordinates',
       'longitude', 'latitude'],
      dtype='object')

In [16]:
def bicimad2(path):
    df = pd.read_csv(path, sep='\t')
    df = df.reset_index(drop=True).loc[:,'name':]
    df.columns = ['BiciMAD station', 'light', 'number', 'Station location', 'activate','no_available', 'total_bases', 'Available bikes', 'Available docks', 'reservations_count', 'geometry.type', 'geometry.coordinates']
    df[['longitude', 'latitude']] = df['geometry.coordinates'].str.strip('[]').str.split(',',expand= True).astype('float64')
    print('Bicimad dataframe created')
    return df

In [17]:
bicimad_df2 = bicimad2('../data/origin/bicimad_stations.csv')
bicimad_df2.head()

Bicimad dataframe created


,BiciMAD station,light,number,Station location,activate,no_available,total_bases,Available bikes,Available docks,reservations_count,geometry.type,geometry.coordinates,longitude,latitude
0,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]",-3.701834,40.417214
1,1b - Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.701602938060457, 40.41731271011562]",-3.701603,40.417313
2,2 - Miguel Moya,3,2,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,"[-3.7058415, 40.4205886]",-3.705842,40.420589
3,3 - Plaza Conde Suchil,2,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,"[-3.7069171, 40.4302937]",-3.706917,40.430294
4,4 - Malasaña,1,4,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,"[-3.7025875, 40.4285524]",-3.702587,40.428552


In [13]:
def bicimad():
    login_url = 'https://openapi.emtmadrid.es/v1/mobilitylabs/user/login/'
    login_headers = {'email': emt_email, 'password': emt_password}
    login_response = requests.get(login_url,headers=login_headers)
    token = login_response.json()['data'][0]['accessToken']
    bicimad_headers = {'accessToken': token}
    bicimad_url = 'https://openapi.emtmadrid.es/v1/transport/bicimad/stations/'
    bicimad_response = requests.get(bicimad_url, headers=bicimad_headers)
    df = pd.json_normalize(bicimad_response.json()['data'])
    df.columns = ['activate', 'Station location', 'Available bikes', 'Available docks', 'id', 'light','BiciMAD station', 'no_available', 'number', 'reservations_count', 'total_bases', 'virtualDelete', 'geofenced_capacity', 'geometry.type', 'geometry.coordinates']
    df['longitude'] = df['geometry.coordinates'].apply(lambda x: x[0]).astype('float64')
    df['latitude'] = df['geometry.coordinates'].apply(lambda x: x[1]).astype('float64')
    df['BiciMAD station'] = [station.split('- ')[-1] for station in df['BiciMAD station']]
    print('Bicimad dataframe created')
    return df

In [18]:
bicimad_df = bicimad()
bicimad_df.head()

Bicimad dataframe created


,activate,Station location,Available bikes,Available docks,id,light,BiciMAD station,no_available,number,reservations_count,total_bases,virtualDelete,geofenced_capacity,geometry.type,geometry.coordinates,longitude,latitude
0,1,"Avenida del Ensanche de Vallecas, 9,",9,18,2190,2,"Avenida del Ensanche de Vallecas, 9",0,453,0,27,False,20.0,Point,"[-3.61253028, 40.37043968]",-3.612530,40.370440
1,1,"Paseo de la Chopera,33,Comunidad de Madrid España",19,4,2205,1,"Paseo de la Chopera, 33",0,267,0,23,False,20.0,Point,"[-3.7003996647238746, 40.39500000000003]",-3.700400,40.395000
2,1,"Paseo de la Castellana nº 122,",1,22,2224,0,Castellana frente a Hermanos Pinzón,0,150,0,23,False,20.0,Point,"[-3.6908, 40.4491]",-3.690800,40.449100
3,1,"Paseo de la Castellana nº 164,Comunidad de Mad...",2,21,2225,0,Castellana 164,0,157,0,23,False,20.0,Point,"[-3.6894151, 40.4591366]",-3.689415,40.459137
4,1,"Guetaria , 84b,",19,5,2226,1,Guetaria 84b,0,395b,0,24,False,20.0,Point,"[-3.71569102, 40.36916772]",-3.715691,40.369168


In [24]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def fuzzy_check(str,places):
    fuzzy_choice = process.extractOne(str, places, scorer=fuzz.token_sort_ratio)
    if fuzzy_choice[1] > 90:
        return fuzzy_choice[0]
    else:
        print(f"{str} doesn't match any monument in the database")

print(bicimad_df2['Station location'].describe())
len([fuzzy_check(location,list(bicimad_df['Station location'])) for location in bicimad_df2['Station location']])


count                     264
unique                    257
top       Puerta del Sol nº 1
freq                        2
Name: Station location, dtype: object
Puerta del Sol nº 1 doesn't match any monument in the database
Puerta del Sol nº 1 doesn't match any monument in the database
Calle Duque de Liria doesn't match any monument in the database
Calle San Vicente Ferrer nº 64 doesn't match any monument in the database
Paseo de la Infanta Isabel nº 3 doesn't match any monument in the database
Paseo de la Infanta Isabel nº 3 doesn't match any monument in the database
Avenida de Menéndez Pelayo esquina calle O'Donnell doesn't match any monument in the database
Calle Embajadores nº 101 doesn't match any monument in the database
Calle León Gil de Palacio nº 3 doesn't match any monument in the database
Glorieta Rubén Darío 2 doesn't match any monument in the database
Calle Francisco Balseiro nº 1 doesn't match any monument in the database


264

In [158]:
def interest_simplification(df):
    df['Type of place'] = 'Monument'
    df=df[['title', 'Type of place', 'address.street-address']]
    df.columns = ['Place of interest', 'Type of place', 'Place address']
    return df

def nearest_all(df1, df2, interest_point):
    i = 1
    latlon = (df1['location.latitude'][i], df1['location.longitude'][i])
    dist = [gd(latlon,(df2['latitude'][i], df2['longitude'][i])).m for i in range(len(df2['longitude']))]
    nearest_station_df = df2.loc[dist.index(min(dist)),['BiciMAD station','Station location','Available bikes', 'Available docks']]._append(pd.Series({'Distance to station (meters)' : round(min(dist),0)}))
    ps = interest_simplification(df1).iloc[i,:]._append(nearest_station_df)
    print(f'{interest_point} added to dataframe')
    return ps


In [165]:
nearest_all(interest_df, bicimad_df2, 'interest_point')

interest_point added to dataframe


Place of interest                               A los Abuelos
Type of place                                        Monument
Place address                                       C Alicún 
BiciMAD station                         249 - Manuel Caldeiro
Station location                Paseo de la Castellana nº 298
Available bikes                                            11
Available docks                                            11
Distance to station (meters)                           2002.0
dtype: object

In [154]:
i = 1
latlon = (interest_df['location.latitude'][i], interest_df['location.longitude'][i])
dist = [gd(latlon,(bicimad_df2['latitude'][i], bicimad_df2['longitude'][i])).m for i in range(len(bicimad_df2['longitude']))]
print(min(dist))
dist.index(min(dist))

2001.790816900556


251

In [10]:
from dotenv import dotenv_values

emt_email = dotenv_values('../.env')['EMT_EMAIL']
emt_password = dotenv_values('../.env')['EMT_PASSWORD']

In [121]:
def interest_points():
    base_url = 'https://datos.madrid.es/egob'
    url = '/catalogo/300356-0-monumentos-ciudad-madrid.json'
    df = pd.json_normalize(requests.get(base_url + url).json()['@graph'])
    print('Interest points dataframe created')
    return df


def bicimad():
    login_url = 'https://openapi.emtmadrid.es/v1/mobilitylabs/user/login/'
    login_headers = {'email': emt_email, 'password': emt_password}
    login_response = requests.get(login_url,headers=login_headers)
    token = login_response.json()['data'][0]['accessToken']
    bicimad_headers = {'accessToken': token}
    bicimad_url = 'https://openapi.emtmadrid.es/v1/transport/bicimad/stations/'
    bicimad_response = requests.get(bicimad_url, headers=bicimad_headers)
    df = pd.json_normalize(bicimad_response.json()['data'])
    df.columns = ['activate', 'Station location', 'Available bikes', 'Available docks', 'id', 'light','BiciMAD station', 'no_available', 'number', 'reservations_count', 'total_bases', 'virtualDelete', 'geofenced_capacity', 'geometry.type', 'geometry.coordinates']
    df['longitude'] = df['geometry.coordinates'].apply(lambda x: x[0]).astype('float64')
    df['latitude'] = df['geometry.coordinates'].apply(lambda x: x[1]).astype('float64')
    df['BiciMAD station'] = [station.split('- ')[-1] for station in df['BiciMAD station']]
    df = df.reset_index(drop=True)
    print('Bicimad dataframe created')
    return df


def nearest(places, bicimad):
    data_list = []
    places_lat_rad = np.radians(places['location.latitude'].to_numpy())
    places_lon_rad = np.radians(places['location.longitude'].to_numpy())
    bicimad_lat_rad = np.radians(bicimad['latitude'].to_numpy())
    bicimad_lon_rad = np.radians(bicimad['longitude'].to_numpy())
    dlat = bicimad_lat_rad[:, np.newaxis] - places_lat_rad
    dlon = bicimad_lon_rad[:, np.newaxis] - places_lon_rad
    
    a = np.sin(dlat / 2) ** 2 + np.cos(bicimad_lat_rad[:, np.newaxis]) * np.cos(places_lat_rad) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    distance_matrix = c * 6371000 

    min_distance_indices = np.argmin(distance_matrix, axis=0)

    """ Creación del dataframe del resultado utilizando ese índice"""

    for x in range(len(places["title"])):
        station_index = min_distance_indices[x]
        station = bicimad['BiciMAD station'].iloc[station_index].split('- ')[-1]
        station_address = bicimad["Station location"].iloc[station_index]
        type_of_place = 'Monument'
        place_address = places["address.street-address"][x]
        place = places["title"][x]
        available_bikes = bicimad['Available bikes'].iloc[station_index]
        available_docks = bicimad['Available docks'].iloc[station_index]
        min_distance = round(distance_matrix[station_index, x], 2)
        data_list.append({"Place of interest": place,'Type of place': type_of_place, "Place address": place_address, "BiciMAD station": station, "Station location": station_address, 'Available bikes': available_bikes, 'Available docks': available_docks, "distance": min_distance})
    df = pd.DataFrame(data_list).set_index('Place of interest')
    df.to_csv('../data/output/1_all_nearest_stations.csv')
    return df

In [122]:
interest_df = interest_points()
bicimad_df2 = bicimad()
all_df = nearest(interest_df, bicimad_df2)
all_df.head()

Interest points dataframe created
Bicimad dataframe created


,Type of place,Place address,BiciMAD station,Station location,Available bikes,Available docks,distance
Place of interest,,,,,,,
A las víctimas del Holocausto,Monument,JARDIN TRES CULTURAS,"Avda del Partenon, 16","564 - Avda del Partenon, 16,",1,22,873.65
A los Abuelos,Monument,C Alicún,"Calle de Bacares, 20","Calle de Bacares, 20,",11,13,349.70
A Neruda-El Ser alado,Monument,Gta José Luis Ozores 116,"433 -Calle de Villalobos, 28","Calle de Villalobos, 28,",6,18,142.94
Abogados Laboristas,Monument,C Atocha 55,Antón Martín,"Calle Atocha nº 54,",3,16,109.52
Abstracta I,Monument,,Quinta Fuente del Berro,Calle Sancho Dávila esquina Paseo Marqués de Z...,20,4,376.75


In [127]:
def stations_coordinates(df1,df2):
    stations = list(df1['BiciMAD station'])
    latitudes = [df2.loc[df2['BiciMAD station'] == station,['latitude']].iloc[0,0] for station in stations]
    longitudes = [df2.loc[df2['BiciMAD station'] == station,['longitude']].iloc[0,0] for station in stations]
    return list(zip(latitudes,longitudes))

In [128]:
stations_coordinates(all_df.iloc[:5,:],bicimad_df2)

[(40.463861, -3.613867),
 (40.47675499, -3.66269029),
 (40.38161669, -3.65076037),
 (40.4122047, -3.6991147),
 (40.4250833, -3.6615277)]

In [31]:
from datetime import datetime, timedelta
import requests
from dotenv import dotenv_values
import polyline
import folium
import matplotlib.image as mpimg

token_gmap = dotenv_values('../.env')['CLAVE_GMAP']

In [29]:
str(datetime.now() + timedelta(days=1)).replace(' ','T')

'2023-11-14T21:14:48.174161'

In [155]:
route_point_coordinates = [(bicimad_df2['latitude'][i],bicimad_df2['longitude'][i]) for i in range(5)]

def route_optimizer(route_point_coordinates):
  route_url = 'https://routes.googleapis.com/directions/v2:computeRoutes' 
  
  data={"origin":{
      "location":{
        "latLng":{
          "latitude": route_point_coordinates[0][0],
          "longitude": route_point_coordinates[0][1]
        }
      }
    },
    "destination":{
      "location":{
        "latLng":{
          "latitude": route_point_coordinates[-1][0],
          "longitude": route_point_coordinates[-1][1]
        }
      }
    },
    "optimizeWaypointOrder": "true",
    "travelMode": "DRIVE",
    "languageCode": "en-US",
    "units": "IMPERIAL"
  } 
  data['intermediates'] = [{'location':{'latLng':{'latitude': route_point_coordinates[i][0],'longitude': route_point_coordinates[i][0]}}} for i in range(1,4)]
  headers = {'Content-Type': 'application/json','X-Goog-Api-Key':f'{token_gmap}','X-Goog-FieldMask': 'routes.optimizedIntermediateWaypointIndex'}

  response = requests.post(route_url, headers=headers, json=data)
  indexes = [i+1 for i in response.json()['routes'][0]['optimizedIntermediateWaypointIndex']]
  return [route_point_coordinates[0]] + [route_point_coordinates[i] for i in indexes] + [route_point_coordinates[-1]]

In [156]:
stations_coordinates= route_optimizer(route_point_coordinates)
stations_coordinates

[(40.37043968, -3.61253028),
 (40.4591366, -3.6894151),
 (40.4491, -3.6908),
 (40.39500000000003, -3.7003996647238746),
 (40.36916772, -3.71569102)]

In [177]:
start_end = [(40.419752, -3.688398), (40.47675499, -3.66269029)]
def section_generator(start,end):
  route_url = 'https://routes.googleapis.com/directions/v2:computeRoutes' 

  data={"origin":{
      "location":{
        "latLng":{
          "latitude": start[0],
          "longitude": start[1]
        }
      }
    },
    "destination":{
      "location":{
        "latLng":{
          "latitude": end[0],
          "longitude": end[1]
        }
      }
    },
    "travelMode": "BICYCLE",
    "languageCode": "en-US",
    "units": "IMPERIAL"
  }
  headers = {'Content-Type': 'application/json','X-Goog-Api-Key':f'{token_gmap}','X-Goog-FieldMask': 'routes.duration,routes.distanceMeters,routes.polyline.encodedPolyline'}

  response = requests.post(route_url, headers=headers, json=data)
  return polyline.decode(response.json()['routes'][0]['polyline']['encodedPolyline'])
  #return response.json()

In [176]:
section_generator(start_end[0], start_end[1])

{'routes': [{'distanceMeters': 8764,
   'duration': '1993s',
   'polyline': {'encodedPolyline': 'knuuFlkoUKMb@yA]SGF_@EMg@u@kGBEGa@@CKAeAqI}@oJGo@JMFCYmB_@gAc@_EDQa@a@EY_FrAiI_@iW_AuKa@oSu@iGSkXgAeNw@oP{@c@Gk[gBe@Lo`@yBOIgBGe@KsJg@h@k`@CaBKkA[eB_@mAWo@w@sAy@{@a@Yk@[@WwA]Aa@K]QO]EUHSZE^kACArA]b@kACCvBC\\_@HuAx@oA|@e@JoD@i@JcEhB_@HoF\\q@T}@`@e@Ta@^BaA`@qGUGSQi@M[OM?AJM@w@eIMkBDEM_@QoAKYYmDmA\\sAh@WFCm@e@B}Cr@eMxBa@J_@@yBuVAm@Ic@U]u@i@Y?_@j@qChCe@u@e@q@uAnBaAlAe@RkANQBKQe@k@kAkCJS@YIUQKSBCDQe@MOWIcADb@r@'}}]}

In [178]:
route_coordinates = []
for i in range(len(stations_coordinates)-1):
    route_coordinates +=section_generator(stations_coordinates[i],stations_coordinates[i+1])
route_coordinates

[(40.37041, -3.61256),
 (40.3708, -3.61336),
 (40.37087, -3.6134),
 (40.37095, -3.61354),
 (40.37098, -3.61362),
 (40.37096, -3.61379),
 (40.3709, -3.61392),
 (40.371, -3.6141),
 (40.37118, -3.61394),
 (40.37128, -3.61395),
 (40.37136, -3.61392),
 (40.37142, -3.61397),
 (40.37166, -3.61429),
 (40.37183, -3.61446),
 (40.37204, -3.61464),
 (40.37239, -3.61487),
 (40.37271, -3.61502),
 (40.37303, -3.61511),
 (40.37343, -3.61515),
 (40.37356, -3.61515),
 (40.37363, -3.61523),
 (40.37364, -3.61528),
 (40.37377, -3.61528),
 (40.37398, -3.61512),
 (40.37403, -3.61512),
 (40.37456, -3.6149),
 (40.37463, -3.61494),
 (40.37465, -3.61497),
 (40.37489, -3.61477),
 (40.37507, -3.6147),
 (40.37868, -3.61328),
 (40.37871, -3.6132),
 (40.37874, -3.61311),
 (40.37881, -3.61307),
 (40.37888, -3.6131),
 (40.3789, -3.61313),
 (40.37902, -3.61306),
 (40.37935, -3.613),
 (40.37935, -3.61288),
 (40.37945, -3.61286),
 (40.37938, -3.61247),
 (40.37955, -3.61242),
 (40.37954, -3.61227),
 (40.37954, -3.61223),
 

In [204]:
def interest_points():
    base_url = 'https://datos.madrid.es/egob'
    url = '/catalogo/300356-0-monumentos-ciudad-madrid.json'
    df = pd.json_normalize(requests.get(base_url + url).json()['@graph'])
    print('Interest points dataframe created')
    return df
interest_points_df = interest_points()

Interest points dataframe created


In [96]:
def interest_points():
    base_url = 'https://datos.madrid.es/egob'
    url = '/catalogo/300356-0-monumentos-ciudad-madrid.json'
    df = pd.json_normalize(requests.get(base_url + url).json()['@graph'])
    print('Interest points dataframe created')
    return df


In [107]:
interest_df = interest_points()

Interest points dataframe created


In [95]:
all_df = nearest(interest_points_df, bicimad_df2)
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def fuzzy_check(str,places):
    fuzzy_choice = process.extractOne(str, places, scorer=fuzz.token_sort_ratio)
    if fuzzy_choice[1] > 80:
        return fuzzy_choice[0]
    else:
        print(f"{str} doesn't match any monument in the database")

def route(df):
    places = input("Write the places you want to visit separated by comas: ").split(',')
    target_places = [fuzzy_check(place,df.index) for place in places]
    target_places = [place for place in target_places if place != None]
    df = df.loc[target_places,:]
    return df
route_df = route(all_df)

NameError: name 'nearest' is not defined

In [221]:
m = folium.Map(location=[route_coordinates[0][0], route_coordinates[0][1]], zoom_start=14)

folium.PolyLine(route_coordinates).add_to(m)

icon_image = mpimg.imread('../data/origin/bicimad_icon.webp')

for lat, lon in stations_coordinates:
    icon = folium.CustomIcon(icon_image, icon_size=(28, 28), icon_anchor=(22, 32),)
    folium.Marker(location=[lat, lon], icon=icon, popup="BiciMAD station").add_to(m)

target_interest_df = interest_df.set_index('title').loc[list(route_df.index),]

icon_image = mpimg.imread('../data/origin/monument_icon.png')
for lat, lon in zip(target_interest_df['location.latitude'], target_interest_df['location.longitude']):
    icon = folium.CustomIcon(icon_image, icon_size=(28, 28), icon_anchor=(22, 32),)
    folium.Marker(location=[lat,lon], icon=icon, popup="monument").add_to(m)

m.save('./map.html')
print(stations_coordinates)

[(40.37043968, -3.61253028), (40.4591366, -3.6894151), (40.4491, -3.6908), (40.39500000000003, -3.7003996647238746), (40.36916772, -3.71569102)]


In [217]:
target_interest_df

,title,@id,id,relation,references,address.district.@id,address.locality,address.postal-code,address.street-address,location.latitude,location.longitude,organization.organization-desc,organization.organization-name,address.area.@id
0,Puerta de Alcalá,https://datos.madrid.es/egob/catalogo/tipo/mon...,408415,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,Pza Independencia,40.420024,-3.688727,"La primera Puerta de Alcalá, sustituta tardía ...",Puerta de Alcalá,https://datos.madrid.es/egob/kos/Provincia/Mad...
1,Fuente de Cibeles,https://datos.madrid.es/egob/catalogo/tipo/mon...,408074,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,Pza Cibeles,40.419331,-3.693093,La fuente de Cibeles fue diseñada en 1777 por ...,Fuente de Cibeles,https://datos.madrid.es/egob/kos/Provincia/Mad...
2,A los Abuelos,https://datos.madrid.es/egob/catalogo/tipo/mon...,409512,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,C Alicún,40.479812,-3.661719,"Pequeña escultura, situada en un parque delimi...",A los Abuelos,https://datos.madrid.es/egob/kos/Provincia/Mad...
3,Abstracta I,https://datos.madrid.es/egob/catalogo/tipo/mon...,408920,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,,40.421746,-3.662293,"La histórica quinta de la Fuente del Berro, de...",Abstracta I,https://datos.madrid.es/egob/kos/Provincia/Mad...
4,Neptuno,https://datos.madrid.es/egob/catalogo/tipo/mon...,408085,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,Pza Cánovas del Castillo,40.415252,-3.694160,La fuente de Neptuno formaba parte de la orden...,Neptuno,https://datos.madrid.es/egob/kos/Provincia/Mad...


In [198]:
''' df = df.reset_index(drop=True).loc[:,'name':]
    df.columns = ['BiciMAD station', 'light', 'number', 'Station location', 'activate','no_available', 'total_bases', 'Available bikes', 'Available docks', 'reservations_count', 'geometry.type', 'geometry.coordinates']
    df[['longitude', 'latitude']] = df['geometry.coordinates'].str.strip('[]').str.split(',',expand= True).astype('float64')
    print('Bicimad dataframe created')
    return df'''

bicimad_df2.columns



Index(['activate', 'address', 'dock_bikes', 'free_bases', 'id', 'light',
       'name', 'no_available', 'number', 'reservations_count', 'total_bases',
       'virtualDelete', 'geofenced_capacity', 'geometry.type',
       'geometry.coordinates'],
      dtype='object')

In [91]:
from polyline import decode
def start_end(start,end):
  route_url = 'https://routes.googleapis.com/directions/v2:computeRoutes' 
  token_gmap = dotenv_values('../.env')['CLAVE_GMAP']

  data={"origin":{ "address": start}, 
    "destination":{ "address": end},
    "travelMode": "TRANSIT",
    "languageCode": "en-US",
    "units": "IMPERIAL"
  }
  headers = {'Content-Type': 'application/json','X-Goog-Api-Key':f'{token_gmap}','X-Goog-FieldMask': 'routes.duration,routes.distanceMeters,routes.polyline.encodedPolyline'}

  response = requests.post(route_url, headers=headers, json=data)
   
  coordinates = decode(response.json()['routes'][0]['polyline']['encodedPolyline'])
  return coordinates[0] + coordinates[-1]

In [93]:
start = 'camino de perales 12'
end = 'martinez villergas 52'
origin_destination = section_generator2(start,end)

In [130]:
type(interest_df['location.latitude'][0])


numpy.float64

In [102]:
def start_end():
  start = input('enter your starting address: ')
  end = input ('enter your desired finishing adress: ')
  route_url = 'https://routes.googleapis.com/directions/v2:computeRoutes' 
  token_gmap = dotenv_values('../.env')['CLAVE_GMAP']

  data={"origin":{ "address": start}, 
    "destination":{ "address": end},
    "travelMode": "TRANSIT",
    "languageCode": "en-US",
    "units": "IMPERIAL"
  }
  headers = {'Content-Type': 'application/json','X-Goog-Api-Key':f'{token_gmap}','X-Goog-FieldMask': 'routes.duration,routes.distanceMeters,routes.polyline.encodedPolyline'}

  response = requests.post(route_url, headers=headers, json=data)
   
  coordinates = decode(response.json()['routes'][0]['polyline']['encodedPolyline'])
  return [start] + [coordinates[0]] + [end] + [coordinates[-1]]

In [381]:
def nearby(lat,lon):
  places_url = 'https://places.googleapis.com/v1/places:searchNearby'
  token_gmap = dotenv_values('../.env')['CLAVE_GMAP']
  data={"includedTypes": ['restaurant'],
    "maxResultCount": 10,
    "locationRestriction": {
      "circle": {
        "center": {
          "latitude": lat,
          "longitude": lon},
        "radius": 2500.0
      }
    }
  }
  places_headers = {'Content-Type': 'application/json','X-Goog-Api-Key':f'{token_gmap}','X-Goog-FieldMask': 'places.displayName,places.primaryTypeDisplayName,places.location,places.priceLevel,places.rating,places.userRatingCount,places.websiteUri'}
  return pd.json_normalize(requests.post(places_url,headers=places_headers,json=data).json()['places'])

In [382]:

near_places = nearby(40.37043968, -3.61253028)

In [378]:
list(near_places['websiteUri'])

['https://www.faunia.es/?utm_id=GMBtraffic&utm_medium=OrganicGMB&utm_source=google',
 'http://www.madrid.es/portales/munimadrid/es/Inicio/Contacto/Direcciones-y-telefonos/Parque-de-la-Gavia?vgnextfmt=default&vgnextoid=48a88d736701a410VgnVCM1000000b205a0aRCRD&vgnextchannel=bfa48ab43d6bb410VgnVCM100000171f5a0aRCRD',
 'http://info.igme.es/ielig/LIGInfo.aspx?codigo=TM023',
 'https://www.madrid.es/portales/munimadrid/es/Inicio/El-Ayuntamiento/Parques-y-jardines/Patrimonio-Verde/Parques-en-Madrid/Parque-Vallecas-Villa/?vgnextfmt=default&vgnextoid=e83b9805833ce210VgnVCM1000000b205a0aRCRD&vgnextchannel=38bb1914e7d4e210VgnVCM1000000b205a0aRCRD',
 nan,
 nan,
 nan,
 nan,
 'https://patrimonioypaisaje.madrid.es/sites/v/index.jsp?vgnextoid=6b78091d1b9c4510091d1b9c45102e085a0aRCRD&vgnextchannel=8fac3cb702aa4510VgnVCM1000008a4a900aRCRD',
 nan]

In [373]:
near_places

,location,rating,websiteUri,userRatingCount,displayName,primaryTypeDisplayName
0,"{'latitude': 40.392078, 'longitude': -3.612436}",4.1,https://www.faunia.es/?utm_id=GMBtraffic&utm_m...,21498.0,"{'text': 'Faunia', 'languageCode': 'en'}","{'text': 'Zoo', 'languageCode': 'en-US'}"
1,"{'latitude': 40.366939599999995, 'longitude': ...",4.4,http://www.madrid.es/portales/munimadrid/es/In...,2549.0,"{'text': 'Parque de La Gavia', 'languageCode':...","{'text': 'Park', 'languageCode': 'en-US'}"
2,"{'latitude': 40.3864209, 'longitude': -3.5971634}",4.4,http://info.igme.es/ielig/LIGInfo.aspx?codigo=...,831.0,"{'text': 'Cerro Almodóvar', 'languageCode': 'en'}","{'text': 'Park', 'languageCode': 'en-US'}"
3,"{'latitude': 40.3718584, 'longitude': -3.6244347}",3.9,https://www.madrid.es/portales/munimadrid/es/I...,505.0,"{'text': 'Cataratas Park', 'languageCode': 'en'}","{'text': 'Park', 'languageCode': 'en-US'}"
4,"{'latitude': 40.3722742, 'longitude': -3.62504...",3.8,NaN,418.0,"{'text': 'Parque Navamures', 'languageCode': '...","{'text': 'Park', 'languageCode': 'en-US'}"
5,"{'latitude': 40.3858411, 'longitude': -3.6069763}",3.0,NaN,1.0,"{'text': 'Fuente Castrillo de aza', 'languageC...","{'text': 'Tourist Attraction', 'languageCode':..."
6,"{'latitude': 40.364715200000006, 'longitude': ...",4.3,NaN,99.0,"{'text': 'Open-air Auditorium', 'languageCode'...","{'text': 'Park', 'languageCode': 'en-US'}"
7,"{'latitude': 40.3809107, 'longitude': -3.61800...",NaN,NaN,NaN,"{'text': 'Mural ""Locomotora de Vapor""', 'langu...","{'text': 'Tourist Attraction', 'languageCode':..."
8,"{'latitude': 40.3858919, 'longitude': -3.60697...",5.0,https://patrimonioypaisaje.madrid.es/sites/v/i...,1.0,"{'text': 'Fuente de Santa Eugenia', 'languageC...","{'text': 'Tourist Attraction', 'languageCode':..."
9,"{'latitude': 40.3810587, 'longitude': -3.6181231}",3.0,NaN,1.0,"{'text': 'Franvia', 'languageCode': 'es'}","{'text': 'Tourist Attraction', 'languageCode':..."


In [367]:
pd.DataFrame(near_places)['websiteUri'][5]

nan

In [302]:
[near_places[0]['displayName']['text'], near_places[0]['location']['latitude'], near_places[0]['location']['longitude']]

['Mural ALMA Boa Mistura', 40.367508, -3.6121849]

In [118]:
start_stop = start_end()
type(start_stop[1][0])

float

In [128]:
test = np.radians(start_stop[3][1])
type(test)

numpy.float64

In [105]:
start= pd.Series({'@id':'', 'id':'', 'title':'Starting location', 'relation':'', 'references':'', 'address.district.@id':'',
       'address.locality':'', 'address.postal-code':'', 'address.street-address':start_stop[0],
       'location.latitude': start_stop[1][0], 'location.longitude': start_stop[1][1],
       'organization.organization-desc':'', 'organization.organization-name':'',
       'address.area.@id':''})
stop= pd.Series({'@id':'', 'id':'', 'title':'Finish location', 'relation':'', 'references':'', 'address.district.@id':'',
       'address.locality':'', 'address.postal-code':'', 'address.street-address':start_stop[2],
       'location.latitude': start_stop[3][0], 'location.longitude': start_stop[3][1],
       'organization.organization-desc':'', 'organization.organization-name':'',
       'address.area.@id':''})

In [155]:
test_df = pd.concat([start.to_frame().T,interest_df,stop.to_frame().T], axis=0).reset_index(drop=True)
test_df

,@id,id,title,relation,references,address.district.@id,address.locality,address.postal-code,address.street-address,location.latitude,location.longitude,organization.organization-desc,organization.organization-name,address.area.@id
0,,,Starting location,,,,,,camino de perales 12,40.38217,-3.6936,,,
1,https://datos.madrid.es/egob/catalogo/tipo/mon...,409634,A las víctimas del Holocausto,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,JARDIN TRES CULTURAS,40.464322,-3.603558,"La obra, creada por el escultor judío de orige...",A las víctimas del Holocausto,NaN
2,https://datos.madrid.es/egob/catalogo/tipo/mon...,409512,A los Abuelos,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,C Alicún,40.479812,-3.661719,"Pequeña escultura, situada en un parque delimi...",A los Abuelos,https://datos.madrid.es/egob/kos/Provincia/Mad...
3,https://datos.madrid.es/egob/catalogo/tipo/mon...,408781,A Neruda-El Ser alado,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,Gta José Luis Ozores 116,40.381855,-3.652419,Monumento dedicado al escritor Pablo Neruda (1...,A Neruda-El Ser alado,https://datos.madrid.es/egob/kos/Provincia/Mad...
4,https://datos.madrid.es/egob/catalogo/tipo/mon...,408836,Abogados Laboristas,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,C Atocha 55,40.412911,-3.700017,El 24 de enero de 1977 se produjo un atentado ...,Abogados Laboristas,https://datos.madrid.es/egob/kos/Provincia/Mad...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1829,https://datos.madrid.es/egob/catalogo/tipo/mon...,408375,Ángeles de la Paz,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,Avda Bruselas,40.443023,-3.664153,"Este monumento, proyectado en chapa de hierro ...",Ángeles de la Paz,https://datos.madrid.es/egob/kos/Provincia/Mad...
1830,https://datos.madrid.es/egob/catalogo/tipo/mon...,409037,Árbol de la Vida,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,JARDIN TRES CULTURAS,40.464322,-3.603558,El Jardín de las Tres Culturas es un ámbito es...,Árbol de la Vida,NaN
1831,https://datos.madrid.es/egob/catalogo/tipo/mon...,408008,Íñigo Arista,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,Pza Oriente,40.41771,-3.712172,Forma parte esta escultura de la serie concebi...,Íñigo Arista,https://datos.madrid.es/egob/kos/Provincia/Mad...
1832,https://datos.madrid.es/egob/catalogo/tipo/mon...,400117,Órgano,https://patrimonioypaisaje.madrid.es/sites/v/i...,https://patrimonioypaisaje.madrid.es/FrameWork...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,,C Castelló 77,40.431113,-3.681166,En 1971 el arquitecto José Luis Picardo ganó e...,Órgano,https://datos.madrid.es/egob/kos/Provincia/Mad...


In [153]:
test_df['location.latitude'] = test_df['location.latitude'].to_numpy()
np.radians(test_df['location.latitude'])

TypeError: loop of ufunc does not support argument 0 of type float which has no callable radians method

In [248]:

token_trip = dotenv_values('../.env')['CLAVE_TRIP']
trip_url = f"https://api.content.tripadvisor.com/api/v1/location/nearby_search" 
trip_headers = {"accept": "application/json"}
data= {'latLong': f'{start_stop[1][0]},{start_stop[1][1]}', 'key': '9FF1DC56FF1B42DEBD15F0B3059D532D', 'category': 'attractions','language': 'en'}
response = requests.get(trip_url,headers=trip_headers,params=data)

In [249]:
response.text


'{\n  "error": {\n    "message": "invalid client key: 9FF1DC56FF1B42DEBD15F0B3059D532D-mapper",\n    "type": "UnauthorizedException",\n    "code": "160"\n  }\n}'

In [239]:
token_trip

'9FF1DC56FF1B42DEBD15F0B3059D532D'

In [ ]:
import smtplib
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

port = 587 
smtp_server = "live.smtp.mailtrap.io"
login = "1a2b3c4d5e6f7g" # paste your login generated by Mailtrap
password = dotenv_values('../.env')['CLAVE_MAIL']

subject = "An example of boarding pass"
sender_email = "mailtrap@example.com"
receiver_email = "new@example.com"

message = MIMEMultipart()
message["From"] = sender_email
message["To"] = receiver_email
message["Subject"] = subject

# Add body to email
body = "This is an example of how you can send a boarding pass in attachment with Python"
message.attach(MIMEText(body, "plain"))

filename = "yourBP.pdf"
# Open PDF file in binary mode

# We assume that the file is in the directory where you run your Python script from
with open(filename, "rb") as attachment:
    # The content type "application/octet-stream" means that a MIME attachment is a binary file
    part = MIMEBase("application", "octet-stream")
    part.set_payload(attachment.read())

# Encode to base64
encoders.encode_base64(part)

# Add header 
part.add_header(
    "Content-Disposition",
    f"attachment; filename= {filename}",
)

# Add attachment to your message and convert it to string
message.attach(part)
text = message.as_string()

# send your email
with smtplib.SMTP("live.smtp.mailtrap.io", 587) as server:
    server.login(login, password)
    server.sendmail(
        sender_email, receiver_email, text
    )
print('Sent')

In [413]:
from flask import Flask
from flask import render_template
from flask import request


app = Flask(__name__)
@app.route('/',methods= ['POST','GET'])
def index():
    if request.method == 'POST':
        email = request.form
        print(email)
        subprocess.run('C:/Users/rjcol/miniconda3/envs/pm1/python.exe c:/Users/rjcol/ironhack/project/project_module_1/ih_datamadpt0923_project_m1/main.py -r -s "calle pez, 2" -f "Calle de tetuan" -p "fuente de cibeles, neptuno, Maestro alonso, La gloria y los pegasos, cabo naval, El vecino curioso"')
        return render_template('base.html')
    else:
        return render_template('base.html')

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
[2023-11-18 16:08:00,702] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "c:\Users\rjcol\miniconda3\envs\pm1\Lib\site-packages\flask\app.py", line 1455, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\rjcol\miniconda3\envs\pm1\Lib\site-packages\flask\app.py", line 869, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\rjcol\miniconda3\envs\pm1\Lib\site-packages\flask\app.py", line 867, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\rjcol\miniconda3\envs\pm1\Lib\site-packages\flask\app.py", line 852, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\rjcol\AppData\Lo

ImmutableMultiDict([('email', ''), ('start', ''), ('finish', ''), ('monuments', '')])


In [416]:
from subprocess import Popen, PIPE

process = Popen(['Python', '../main.py'], stdout=PIPE, stderr=PIPE)

stdout, stderr = process.communicate()

print(stdout)

b''


In [447]:
from subprocess import call, run

run(['../../../../../miniconda3/envs/pm1/python.exe','../main.py'],capture_output=True,text=True)

CompletedProcess(args=['../../../../../miniconda3/envs/pm1/python.exe', '../main.py'], returncode=1, stdout='', stderr='Traceback (most recent call last):\n  File "c:\\Users\\rjcol\\ironhack\\project\\project_module_1\\ih_datamadpt0923_project_m1\\main.py", line 44, in <module>\n    main()\n  File "c:\\Users\\rjcol\\ironhack\\project\\project_module_1\\ih_datamadpt0923_project_m1\\main.py", line 21, in main\n    os.mkdir(f\'./data/output/{folder}\')\nFileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: \'./data/output/2023-11-18_171655\'\n')

In [446]:
print(result.stdout)
stderr

b'Traceback (most recent call last):\r\n  File "c:\\Users\\rjcol\\ironhack\\project\\project_module_1\\ih_datamadpt0923_project_m1\\main.py", line 4, in <module>\r\n    import modules.dataframe_creation as dc\r\n  File "c:\\Users\\rjcol\\ironhack\\project\\project_module_1\\ih_datamadpt0923_project_m1\\modules\\dataframe_creation.py", line 5, in <module>\r\n    emt_email = dotenv_values(\'./.env\')[\'EMT_EMAIL\']\r\n                ~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^\r\nKeyError: \'EMT_EMAIL\'\r\n'